## Voice Style Control Demo

In [5]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

### Initialization

In [6]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

/home/bbbs/anaconda3/envs/openVoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average of multiple sentences generated by the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [7]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

The `reference_speaker.mp3` below points to the short audio clip of the reference whose voice we want to clone. We provide an example here. If you use your own reference speakers, please **make sure each speaker has a unique filename.** The `se_extractor` will save the `targeted_se` using the filename of the audio and **will not automatically overwrite.**

In [8]:
reference_speaker = 'resources/ram_veer.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

OpenVoice version: v1
[(0.75, 14.354), (16.078, 29.65), (30.83, 45.682), (46.542, 64.434), (64.75, 87.872)]
after vad: dur = 83.042


/home/bbbs/anaconda3/envs/openVoice/lib/python3.9/site-packages/torch/functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608885084/work/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/home/bbbs/anaconda3/envs/openVoice/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608885084/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.

### Inference

In [9]:
save_path = f'{output_dir}/RAMVEER_1.wav'


# Run the base speaker tts

text = "The video talks about Prime Minister Narendra Modi's call for Aatmanirbhar Bharat, a self-reliant India. The captions says that we have been hearing for a century that the 21st century belongs to India. The world order has been changing since the COVID-19 pandemic. When we look at these two eras from India's perspective, it seems that the 21st century can be India's century. This is not just a dream, but it is our responsibility. The current situation in the world teaches us the importance of self-reliance,  an Aatmanirbhar Bharat. Our scriptures also say that self-reliance is the way forward"

# text = "यह वीडियो प्रधानमंत्री नरेंद्र मोदी के आत्मनिर्भर भारत के आह्वान के बारे में बात करता है। कैप्शन में कहा गया है कि हम एक सदी से सुन रहे हैं कि 21वीं सदी भारत की है। कोविड-19 महामारी के बाद से विश्व व्यवस्था बदल रही है। जब हम इन दो युगों को भारत के नजरिए से देखते हैं, तो ऐसा लगता है कि 21वीं सदी भारत की सदी हो सकती है। यह सिर्फ एक सपना नहीं है, बल्कि यह हमारी जिम्मेदारी है। दुनिया की मौजूदा स्थिति हमें आत्मनिर्भरता, आत्मनिर्भर भारत के महत्व को सिखाती है। हमारे शास्त्र भी कहते हैं कि आत्मनिर्भरता ही आगे का रास्ता है।"

src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
The video talks about Prime Minister Narendra Modi's call for Aatmanirbhar Bharat,
a self-reliant India. The captions says that we have been hearing for a century that the 21st century belongs to India.
The world order has been changing since the COVID-19 pandemic. When we look at these two eras from India's perspective,
it seems that the 21st century can be India's century. This is not just a dream,
but it is our responsibility. The current situation in the world teaches us the importance of self-reliance,
an Aatmanirbhar Bharat. Our scriptures also say that self-reliance is the way forward
 > ===========================
ðə ˈvɪdioʊ tɔks əˈbaʊt pɹaɪm ˈmɪnɪstəɹ naɹendɹa* modi's* kɔɫ fəɹ aatmaniɹbhaɹ* bhaɹat*,
 length:87
 length:86
ə self-ɹeliant* ˈɪndiə. ðə ˈkæpʃənz sɪz ðət wi hæv bɪn ˈhiɹɪŋ fəɹ ə ˈsɛntʃəɹi ðət ðə tˈwɛntiˌfəɹst ˈsɛntʃəɹi bɪˈlɔŋz tɪ ˈɪndiə.
 length:127
 length:127
ðə wəɹɫd ˈɔɹdəɹ həz bɪn ˈtʃeɪndʒɪŋ sɪns ðə covid-nineteen* pænˈdɛmɪk. wɪn wi 

/home/bbbs/anaconda3/envs/openVoice/lib/python3.9/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608885084/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


**Try with different styles and speed.** The style can be controlled by the `speaker` parameter in the `base_speaker_tts.tts` method. Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering. Note that the tone color embedding need to be updated. The speed can be controlled by the `speed` parameter. Let's try whispering with speed 0.9.

In [10]:
# source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
# save_path = f'{output_dir}/output_whispering.wav'

# # Run the base speaker tts
# text = "This audio is generated by OpenVoice."
# src_path = f'{output_dir}/tmp.wav'
# base_speaker_tts.tts(text, src_path, speaker='whispering', language='English', speed=0.9)

# # Run the tone color converter
# encode_message = "@MyShell"
# tone_color_converter.convert(
#     audio_src_path=src_path, 
#     src_se=source_se, 
#     tgt_se=target_se, 
#     output_path=save_path,
#     message=encode_message)

**Try with different languages.** OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker. We provide an example with a Chinese base speaker here and we encourage the readers to try `demo_part2.ipynb` for a detailed demo.

In [67]:

# ckpt_base = 'checkpoints/base_speakers/ZH'
# base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
# base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

# source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)
# save_path = f'{output_dir}/output_chinese.wav'

# # Run the base speaker tts
# text = "今天天气真好，我们一起出去吃饭吧。"
# src_path = f'{output_dir}/tmp.wav'
# base_speaker_tts.tts(text, src_path, speaker='default', language='Chinese', speed=1.0)

# # Run the tone color converter
# encode_message = "@MyShell"
# tone_color_converter.convert(
#     audio_src_path=src_path, 
#     src_se=source_se, 
#     tgt_se=target_se, 
#     output_path=save_path,
#     message=encode_message)

**Tech for good.** For people who will deploy OpenVoice for public usage: We offer you the option to add watermark to avoid potential misuse. Please see the ToneColorConverter class. **MyShell reserves the ability to detect whether an audio is generated by OpenVoice**, no matter whether the watermark is added or not.